In [1]:
import os
import pandas as pd

dir_ = 'data'
files = [x for x in os.listdir(dir_) if x.endswith('.txt')]

experiments = [
    'heavyside_0_to_100',
    'heavyside_100_to_0_with_highBrake2',
    'heavyside_100_to_0_with_highBrake',
    
    'heavyside_100_to_0_with_lowBrake',
    'speedup_0_100',
    'speedup_100_200',
    'speedup_150_250',
    'test_hall_input_with_aRead',
    'test_hall_input_without_aRead'
]

groups = [
    'heavyside_0_to_100',
    'heavyside_100_to_0',
    'speedup',
    'test_hall_input'
]

In [2]:
data = []
cols = ['group', 'experiment', 'experiment_run']
for filename in files:
    name = filename.replace('.txt', '')
    # only read run #0
    # if name not in experiments:
    #     continue
    print(f'Reading from {filename} ...', end='\r')
    df = pd.read_csv(
        os.path.join(dir_, filename),
        on_bad_lines='skip'
    )
    df['experiment'] = 'unnamed'
    df['experiment_run'] = 0
    
    df['group'] = ''
    
    for group in groups:
        if group in name:
            df['group'] = group
            break
        
    for exp in experiments:
        if exp in name:
            df['experiment'] = exp
            run = name.replace(exp, '').replace('_', '')
            if run != '':
                df['experiment_run'] = int(run)
            break
            
    for c in df.columns:
        if c not in cols:
            cols.append(c)
    data.append(df)

df = pd.concat([pd.DataFrame(columns=cols), *data])
df.columns = [x.replace('[', '').replace(']', '') for x in df.columns]


C:\Users\SwisensAG\AppData\Local\Temp\ipykernel_4876\1394825937.py:9: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [3]:
df['Timestamp_micros'] = pd.to_numeric(df['Timestamp_micros'], errors='coerce')
df = df.loc[~df.Timestamp_micros.isna()]
df['Timestamp'] = df['Timestamp_micros']/1e6

df[['aout_speed_uint8', 'speed_measured_hz', 'bool_brake', 'din_hall0']] = df[['aout_speed_uint8', 'speed_measured_hz', 'bool_brake', 'din_hall0']].apply(pd.to_numeric, errors='coerce')

df['Timestamp_down'] = df['Timestamp']//200e-6*200e-6
df['din_hall0 x20'] = df['din_hall0']*20-30
df['bool_brake x10'] = df['bool_brake']*10

In [4]:
df_plt = df[::1].melt(
    id_vars=['group', 'experiment', 'experiment_run', 'Timestamp',],
    value_vars=['aout_speed_uint8', 'bool_brake x10', 'din_hall0 x20', 'speed_measured_hz'],
    value_name='Data',
    var_name='Data_name'
)
df_plt.head()

,group,experiment,experiment_run,Timestamp,Data_name,Data
0,heavyside_0_to_100,heavyside_0_to_100,0,0.000000,aout_speed_uint8,0.0
1,heavyside_0_to_100,heavyside_0_to_100,0,0.000021,aout_speed_uint8,0.0
2,heavyside_0_to_100,heavyside_0_to_100,0,0.000042,aout_speed_uint8,0.0
3,heavyside_0_to_100,heavyside_0_to_100,0,0.000063,aout_speed_uint8,0.0
4,heavyside_0_to_100,heavyside_0_to_100,0,0.000084,aout_speed_uint8,0.0


In [5]:
df_plt.experiment_run.unique()

array([0, 2, 3, 4, 5], dtype=object)

In [9]:
labels = {
    'Timestamp': 'Timestamp [s]',
    'aout_speed_uint8': 'Out speed [-]',
    'bool_brake x10': 'Out brake [-]',
    'din_hall0 x20': 'In Hall [-]',
    'speed_measured_hz': 'In speed [Hz]',
   
}

runs = [0, 2, 3, 4, 5]

In [11]:
import plotly.express as px

for run in runs:
    for group in groups:
        experiments_ = df_plt[(df_plt.group == group) & (df_plt.experiment_run == run)].experiment.unique()
        if len(experiments_) == 0:
            continue

        df_ = df_plt[df_plt.experiment.isin(experiments_)]
        df_.loc[: 'Data_name'] = df_.loc[: 'Data_name'].replace(labels)
        fig = px.line(
            df_[::50],
            x='Timestamp',
            y='Data',
            color='Data_name',
            facet_row='experiment',
            title=group.replace('_', ' ') +f' Run={ run}',
            height=400*len(experiments_),
            labels=labels
        )
        # fig.show()
        fig.write_html(os.path.join('plots', 'Measurements_'+group+f'_run{run}.html'))

        del fig

C:\Users\SwisensAG\AppData\Local\Temp\ipykernel_4876\212812484.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\SwisensAG\AppData\Local\Temp\ipykernel_4876\212812484.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\SwisensAG\AppData\Local\Temp\ipykernel_4876\212812484.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def format_ax(ax):
    ax.set_xlabel('Timestamp [s]')
    ax.set_ylabel('Data')

for run in runs:
    for group in groups:
        experiments_ = df_plt[(df_plt.group == group) & (df_plt.experiment_run == run)].experiment.unique()
        if len(experiments_) == 0:
            continue
        fig, axs = plt.subplots(len(experiments_),1, figsize=(20, 7*len(experiments_)), squeeze=False)

        for i, experiment in enumerate(experiments_):
            # add nice labels
            df_ = df_plt[df_plt.experiment == experiment]
            df_.loc[: 'Data_name'] = df_.loc[: 'Data_name'].replace(labels)
            sns.lineplot(
                data=df_[::50],
                x='Timestamp',
                y='Data',
                hue='Data_name',
                ax=axs[i][0]
                #height=900
            )

            axs[i][0].grid(True)
            axs[i][0].title.set_text('Experiment: '+experiment.replace('_', ' '))
            format_ax(axs[i][0])
        fig.suptitle(group.replace('_', ' ') +f' Run={run}', fontsize=18)

        plt.savefig(os.path.join('plots', group+f'_run{run}.png'))
        del fig

C:\Users\SwisensAG\AppData\Local\Temp\ipykernel_4876\354418723.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\SwisensAG\AppData\Local\Temp\ipykernel_4876\354418723.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\SwisensAG\AppData\Local\Temp\ipykernel_4876\354418723.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.